In [1]:
%reload_ext autoreload
%autoreload 2

import cv2
import numpy as np
import os
from calibrate_table import calibrate_table_by_aruco
from utils import show
from params import table_aruco_size, K, D
from detection import detect_table_markers_on_image_hsv
from shapely.geometry import Polygon

In [ ]:
# visualize table frame
view = "front"
image_file = "data/table_arucos/0000.jpg"
image = cv2.imread(image_file)
table_frame, _ = calibrate_table_by_aruco(image, view, K, D, table_aruco_size)
if table_frame is None:
    print("Could not detect table")
else:
    camera2table = table_frame.origin2plane()
    rvec, _ = cv2.Rodrigues(camera2table[0:3, 0:3])
    tvec = camera2table[0:3, 3]
    draw = image.copy()
    cv2.drawFrameAxes(draw, K, D, rvec, tvec, 0.1)
    show(draw)

In [3]:
# get table aruco corners in table frame
view = "front"
folder = "data/table_arucos/"
files = sorted(os.listdir(folder))
corners_3d_in_table_all = list()
for f in files:
    image = cv2.imread(folder + f)
    table_frame, corners_3d = calibrate_table_by_aruco(image, view, K, D, table_aruco_size)
    corners_3d = corners_3d[:, 0, 0, :]
    # corners_3d.shape = (n, 3)
    corners_3d_in_table = table_frame.to_plane(corners_3d)
    corners_3d_in_table_all.append(corners_3d_in_table)

corners_3d_in_table_all = np.array(corners_3d_in_table_all)
corners_3d_in_table_mean = corners_3d_in_table_all.mean(axis=0)

In [4]:
corners_3d_in_table_mean.shape
np.save('data/aruco_corners.npy', corners_3d_in_table_mean)

In [ ]:
# get markers coords in table frame
hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV_FULL)
table_markers = detect_table_markers_on_image_hsv(hsv, view)
assert len(table_markers) == 4
# table_markers.shape = (4, 1, 2)

table_markers = cv2.undistortPoints(table_markers, K, D)
table_markers.sort(key=lambda a, b: np.sign(sum(a) - sum(b)))
p = Polygon(table_markers[:, 0, :])
if p.exterior.is_ccw:
    table_markers[[1, 3]] = table_markers[[3, 1]]
    p = Polygon(table_markers[:, 0, :])
assert p.exterior.is_simple and not p.exterior.is_ccw

table_markers = table_markers[:, 0, :]
table_markers = np.hstack((table_markers, np.ones((len(table_markers), 1))))
table_markers_3d = table_frame.intersection_with_plane(table_markers)
table_markers_2d = table_markers_3d[:, :2]

In [ ]:
table_markers_3d
np.save('data/table_markers.npy', table_markers_2d)